## Load Dataset

In [ ]:
import json
import os

roi_folder = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca"
eeg_folder = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca"
index_data = {}
# Modify train_dataset and test_eval as per the new requirement
index_data["train_dataset"] = [str(i) for i in range(1, 201)]
index_data["test_eval"] = [str(i) for i in range(201, 241)]

# Write the updated data to a JSON file
with open("subject_groups.json", "w") as json_file:
    json.dump(index_data, json_file, indent=4)
file_status = {}
for subject in index_data["train_dataset"]:
    roi_path = os.path.join(roi_folder, f"processed_{subject}_ICA_DLtrain.set")
    eeg_path = os.path.join(eeg_folder, f"processed_{subject}_ICA_DLtrain.set")

    roi_exists = os.path.exists(roi_path)
    eeg_exists = os.path.exists(eeg_path)

    file_status[subject] = {
        "roi_exists": roi_exists,
        "eeg_exists": eeg_exists
    }

    if roi_exists == False or eeg_exists == False:
        print(f"{subject}: ROI exists: {roi_exists}, EEG exists: {eeg_exists}")

file_status = {}
for subject in index_data["test_eval"]:
    roi_path = os.path.join(roi_folder, f"processed_{subject}_ICA_DLtrain.set")
    eeg_path = os.path.join(eeg_folder, f"processed_{subject}_ICA_DLtrain.set")

    roi_exists = os.path.exists(roi_path)
    eeg_exists = os.path.exists(eeg_path)

    file_status[subject] = {
        "roi_exists": roi_exists,
        "eeg_exists": eeg_exists
    }

    if roi_exists == False or eeg_exists == False:
        print(f"{subject}: ROI exists: {roi_exists}, EEG exists: {eeg_exists}")


In [ ]:
!pip install pymatreader

In [ ]:
import sys
sys.path.insert(0, './Dataloader')
from SLT_dataloader import EEGROIDataset, SignalDataCollator, EEGROI_fft_Dataset

# Usage example
roi_folder = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca"
eeg_folder = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca"
group_file = "subject_groups.json"

# Create dataset
train_dataset = EEGROI_fft_Dataset(roi_folder, eeg_folder, group_file, "test")
print(f"Total dataset size: {len(train_dataset)}")
# Create dataset
# test_dataset = EEGROIDataset(roi_folder, eeg_folder, group_file, "test_dataset")
# print(f"Total dataset size: {len(test_dataset)}")

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
matplotlib.use('Agg')


# 產生模擬 FFT 數據 (30 通道, 51 頻率點)
np.random.seed(42)
fft_data = np.random.rand(30, 51)  # 假設這是 FFT 結果
# fft_data = train_dataset.__getitem__(0)['src'].numpy()
# print(fft_data[0])

# 頻率軸 (0~50 Hz)
freqs = np.linspace(0, 50, 51)

# 畫圖
plt.figure(figsize=(10, 6))

for i in range(30):
    plt.plot(freqs, fft_data[i], label=f"Ch {i+1}", alpha=0.6)

plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.title("EEG Power Spectrum (30 channels)")
plt.legend(loc="upper right", bbox_to_anchor=(1.2, 1), fontsize=8)
plt.grid(True)
plt.show()

In [ ]:
import h5py
import mne 

file_roi = "G:\\共用雲端硬碟\\CNElab_黎承宣&賴璁毅_EEG_ROI\\A.Dataset\\processed_setfile\\processed_0_ICA_DLtrain.set"
file_eeg = "G:\\共用雲端硬碟\\CNElab_黎承宣&賴璁毅_EEG_ROI\\A.Dataset\\setfile\\0_ICA_DLtrain.set"

with h5py.File(file_roi, 'r') as f:
    if 'roi' in f:
        ROI = f['roi']['source_voxel_data'][:]
        print(ROI.shape)

EEG = mne.io.read_raw_eeglab(file_eeg, preload=True)
print(EEG.info)

## Dataset

In [ ]:
import os
import numpy as np
import h5py
import mne
from torch.utils.data import Dataset

roi_folder = "G:\\共用雲端硬碟\\CNElab_黎承宣&賴璁毅_EEG_ROI\\A.Dataset\\processed_setfile"
eeg_folder = "G:\\共用雲端硬碟\\CNElab_黎承宣&賴璁毅_EEG_ROI\\A.Dataset\\setfile"
roi_folder_ls = [f.split('_')[1] for f in os.listdir(roi_folder) if f.endswith('.set')]

shape_list = []

for subject in roi_folder_ls:
    roi_path = os.path.join(roi_folder, f"processed_{subject}_ICA_DLtrain.set")
    eeg_path = os.path.join(eeg_folder, f"{subject}_ICA_DLtrain.set")
    # Load ROI data
    with h5py.File(roi_path, 'r') as f:
        if 'roi' in f:
            roi_data = f['roi']['source_voxel_data'][:]
            print(roi_data.shape)

    # Load EEG data
    eeg_data = mne.io.read_raw_eeglab(eeg_path, preload=True).get_data()
    print(eeg_data.shape)
    shape_list.append((roi_data.shape, eeg_data.shape))

## Save File Name

In [ ]:
import os
import json
import random

def save_filenames_to_file(folder_path, output_file):
    """
    Reads all filenames in a folder and saves them to a specified file.

    Args:
        folder_path (str): Path to the folder containing files.
        output_file (str): Path to the output file to save filenames.
    """
    # Get all filenames in the folder
    filenames = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Save filenames to a JSON file
    with open(output_file, 'w') as f:
        json.dump(filenames, f, indent=4)

    print(f"Saved {len(filenames)} filenames to {output_file}")

def get_common_subjects(folder1, folder2):
    """
    Finds common subjects between two folders based on filenames.

    Args:
        folder1 (str): Path to the first folder.
        folder2 (str): Path to the second folder.

    Returns:
        list: List of common subjects.
    """

    files1 = set([f.split('_')[0] for f in os.listdir(folder1) if f.endswith('.set')])
    files2 = set([f.split('_')[1] for f in os.listdir(folder2) if f.endswith('.set')])
    print(len(files1), len(files2))
    common_subjects = list(files1.intersection(files2))
    return common_subjects

def split_subjects_into_groups(subjects, num_groups):
    """
    Splits subjects into a specified number of random groups.

    Args:
        subjects (list): List of subject filenames.
        num_groups (int): Number of groups to split into.

    Returns:
        dict: Dictionary where keys are group indices and values are lists of subjects.
    """
    random.shuffle(subjects)
    groups = {i: [] for i in range(num_groups)}
    for idx, subject in enumerate(subjects):
        group_idx = idx % num_groups
        groups[group_idx].append(subject)
    return groups

# Example usage
folder2 = "G:\\共用雲端硬碟\\CNElab_黎承宣&賴璁毅_EEG_ROI\\A.Dataset\\processed_setfile"
folder1 = "G:\\共用雲端硬碟\\CNElab_黎承宣&賴璁毅_EEG_ROI\\A.Dataset\\setfile"
output_file = "common_subjects.json"
num_groups = 20

# Find common subjects
common_subjects = get_common_subjects(folder1, folder2)

# Save common subjects to a file
with open(output_file, 'w') as f:
    json.dump(common_subjects, f, indent=4)
print(f"Saved {len(common_subjects)} common subjects to {output_file}")

# Split common subjects into groups
groups = split_subjects_into_groups(common_subjects, num_groups)
group_file = "subject_groups.json"
with open(group_file, 'w') as f:
    json.dump(groups, f, indent=4)
print(f"Saved subject groups to {group_file}")


In [ ]:
import numpy as np
file_path = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\roi_removal_segment.txt"

def txt_to_dict_with_list(txt_file):
    try:
        with open(txt_file, "r", encoding="utf-8") as file:
            lines = file.readlines()
        
        result = {}
        for line in lines:
            line = line.strip()  # 移除空白與換行
            if not line:  # 跳過空行
                continue
            parts = line.split(",")
            filename = parts[0].strip(".set")
            if len(parts) > 1:
                # 將 index 切割為 list
                index = [int(x) for x in parts[1].strip().split()]
            else:
                index = None
            result[filename] = index

        return result
    except Exception as e:
        print(f"發生錯誤: {e}")
        return None

# 主程式
data_dict = txt_to_dict_with_list(file_path)

# 打印結果
if data_dict:
    print("轉換結果為字典：")
    print(data_dict)

if 87 in data_dict["3_ICA_DLtrain"]:
    print("49")


## Numpy data storage

In [2]:
import os
import mne
import numpy as np
import h5py

data_root = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca"
save_root = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca"
os.makedirs(save_root, exist_ok=True)

subjects = range(202,496)

for subject in subjects:
    roi_path = os.path.join(data_root, f"processed_{subject}_ICA_DLtrain.set")

    # 轉換 ROI
    with h5py.File(roi_path, 'r') as f:
        if 'roi' in f:
            roi_data = f['roi']['source_roi_data'][:]
            print(f"ROI shape: {roi_data.shape}")
            np.save(os.path.join(save_root, f"{subject}_roi.npy"), roi_data)

    # # 轉換 EEG
    raw = mne.io.read_raw_eeglab(roi_path, preload=True)
    eeg_data = raw.get_data()  # (channels, time)
    print(f"EEG shape: {eeg_data.shape}")
    np.save(os.path.join(save_root, f"{subject}_eeg.npy"), eeg_data)

ROI shape: (116, 200, 204)


c:\Users\user\anaconda3\envs\huggingface\Lib\site-packages\pymatreader\utils.py:168: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(


Reading G:\共用雲端硬碟\CNElab_陳昱祺\source localization\test_data\ROI\Desikan_Kilianny_with_3pca\processed_202_ICA_DLtrain.fdt
Reading 0 ... 60499  =      0.000 ...   236.324 secs...


C:\Users\user\AppData\Local\Temp\ipykernel_648\1648554290.py:23: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(roi_path, preload=True)


EEG shape: (30, 60500)
ROI shape: (150, 200, 204)
Reading G:\共用雲端硬碟\CNElab_陳昱祺\source localization\test_data\ROI\Desikan_Kilianny_with_3pca\processed_203_ICA_DLtrain.fdt
Reading 0 ... 77281  =      0.000 ...   301.879 secs...
EEG shape: (30, 77282)
ROI shape: (131, 200, 204)
Reading G:\共用雲端硬碟\CNElab_陳昱祺\source localization\test_data\ROI\Desikan_Kilianny_with_3pca\processed_204_ICA_DLtrain.fdt
Reading 0 ... 68445  =      0.000 ...   267.363 secs...


...

C:\Users\user\AppData\Local\Temp\ipykernel_648\1648554290.py:23: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(roi_path, preload=True)


EEG shape: (30, 61237)
ROI shape: (125, 200, 204)
Reading G:\共用雲端硬碟\CNElab_陳昱祺\source localization\test_data\ROI\Desikan_Kilianny_with_3pca\processed_254_ICA_DLtrain.fdt
Reading 0 ... 65372  =      0.000 ...   255.359 secs...


C:\Users\user\AppData\Local\Temp\ipykernel_648\1648554290.py:23: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(roi_path, preload=True)


EEG shape: (30, 65373)
ROI shape: (179, 200, 204)
Reading G:\共用雲端硬碟\CNElab_陳昱祺\source localization\test_data\ROI\Desikan_Kilianny_with_3pca\processed_255_ICA_DLtrain.fdt
Reading 0 ... 91750  =      0.000 ...   358.398 secs...
EEG shape: (30, 91751)
ROI shape: (151, 200, 204)
Reading G:\共用雲端硬碟\CNElab_陳昱祺\source localization\test_data\ROI\Desikan_Kilianny_with_3pca\processed_256_ICA_DLtrain.fdt
Reading 0 ... 77567  =      0.000 ...   302.996 secs...
EEG shape: (30, 77568)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'G:\共用雲端硬碟\CNElab_陳昱祺\source localization\test_data\ROI\Desikan_Kilianny_with_3pca\processed_257_ICA_DLtrain.set', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:

data_root = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca"
subjects = range(0,10)
for subject in subjects:
    roi_path = os.path.join(data_root, f"{subject}_roi.npy")
    eeg_path = os.path.join(data_root, f"{subject}_eeg.npy")

    # 讀取 ROI & EEG
    roi_data = np.load(roi_path)
    eeg_data = np.load(eeg_path)

    assert roi_data.shape[1] == 200, f"Unexpected ROI shape: {roi_data.shape}"
    assert eeg_data.shape[0] == 30, f"Unexpected EEG shape: {eeg_data.shape}"

    # 直接處理資料
    # _process_subject_data(roi_data, eeg_data, subject_name=f"{subject}_ICA_DLtrain")


## Simulate Dataloader Test

In [ ]:
import os
import numpy as np
import mne
import torch
from matplotlib import pyplot as plt

def channel_mapping(_64_ch_names, _32_ch_names, eeg_data):
    _32_eeg_data = []
    
    for ch in _32_ch_names:
        current_idx = _64_ch_names.index(ch)  # 找到 64 通道中的對應索引
        _32_eeg_data.append(eeg_data[current_idx])  # 提取相應數據
    
    return np.array(_32_eeg_data)  # 轉為 NumPy 陣列

def print_channel(_64_ch_names, _32_ch_names, _64_eeg_data, _32_eeg_data):
    flag = True
    for ind, chn in enumerate(_32_ch_names):
        mapping_index = _64_ch_names.index(chn)
        # print(ind, chn, mapping_index, len(_64_eeg_data))
        if _32_eeg_data[ind].sum() != _64_eeg_data[mapping_index].sum():
            flag = False

    return flag

data_path = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\simulate_data\\dataset_100_1000000.0_20250322_152651\\"

# 讀取全部資料
all_eeg_data = np.load(os.path.join(data_path, "all_eeg_data.npy"), allow_pickle=True)
all_source_data = np.load(os.path.join(data_path, "all_source_data.npy"), allow_pickle=True)

_EEG = torch.tensor(np.stack(all_eeg_data), dtype=torch.float32)  # (N, 64, 51)
ROI = torch.tensor(np.stack(all_source_data), dtype=torch.float32)  # (N, 204, 51)

# 建立 channel map (只做一次)
ch_names_64 = mne.io.read_raw('G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_raw.fif', verbose='Warning').resample(100).info['ch_names']
ch_names_32 = ['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2', 'FC6', 
                'T7', 'C3', 'Cz', 'C4', 'T8', 'CP5', 'CP1', 'CP2', 'CP6', 'P7', 'P3', 
                'Pz', 'P4', 'P8', 'POz', 'O1', 'Oz', 'O2']

ch_idx_map = [ch_names_64.index(ch) for ch in ch_names_32]
print(f"EEG = {_EEG.shape}")

# 選出 32 channels
EEG = _EEG[:, ch_idx_map, :]  # (N, 30, 51)

print(f"Preprocessing complete. Total valid samples: {_EEG.shape}")
print(f"Preprocessing complete. Total valid samples: {EEG.shape}")

flag_ = True
for idx, _eeg_ in enumerate(EEG):

    print(_eeg_.shape)

    temp_flag = print_channel(ch_names_64, ch_names_32, _EEG[idx,:,:], _eeg_)
    print(f"Data:{idx} is {temp_flag}")
    flag_ = flag_ & temp_flag

    if idx == 20:
        break

print(flag_)

In [ ]:
import os
# "../../source localization/simulate_data/dataset_25000_1000000.0_20250321_194857/"

source_dataset_path = "../../source localization/simulate_data/dataset_25000_1000000.0_20250321_194857/"
destination_dataset_path = "../../source localization/simulate_data/dataset_25000_1000000.0_20250321_194857/train_dataset/"
lsdir = os.listdir(source_dataset_path)

# 過濾出以 "eeg_data_" 開頭的檔案
eeg_files = [file for file in lsdir if file.startswith("eeg_data_")]

max_idx = 3500
for idx, eeg_path in enumerate(eeg_files):
    source_path = eeg_path.replace("eeg", "source")
    os.replace(source_dataset_path + eeg_path, destination_dataset_path + eeg_path)
    os.replace(source_dataset_path + source_path, destination_dataset_path + source_path)

    # if idx == max_idx:
    #     break

In [ ]:
import sys
sys.path.insert(0, './Dataloader')
from SLT_dataloader import EEGROIDataset, SignalDataCollator, EEGROI_fft_Dataset, EEGROI_Power_Dataset

data_path = "G:\共用雲端硬碟\CNElab_陳昱祺\source localization\simulate_data\\dataset_1000_1000000.0_20250322_225032\\test_dataset\\"

dataset = EEGROI_Power_Dataset(data_path)

print(dataset.__getitem__(0)['src'].shape)
print(dataset.__getitem__(0)['label'].shape)


In [ ]:
import os

folders = [
    "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\simulate_data\\dataset_1000_1_20250323_150519",
    "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\simulate_data\\dataset_1000_1000000.0_20250322_225032",
    "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\simulate_data\\dataset_1000_10_20250324_031827",
]

for folder_path in folders:
    folder_path = os.path.normpath(folder_path)  # 正規化路徑，移除末尾斜線
    folder_name = os.path.basename(folder_path)
    
    print(f"Folder name: {folder_name}")  # 確認你拿到的名稱是對的

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if os.path.isdir(file_path):
            continue

        if filename.startswith(folder_name + "_"):
            continue

        new_filename = f"{folder_name}_{filename}"
        new_file_path = os.path.join(folder_path, new_filename)

        os.rename(file_path, new_file_path)
        print(f"Renamed: {filename} -> {new_filename}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 取得一筆資料
sample = dataset.__getitem__(0)
eeg_power = sample['src']      # [30, 51]
source_power = sample['label'] # [204, 51]

### 畫 EEG Power ###
eeg_ymin = eeg_power.min().item()
eeg_ymax = eeg_power.max().item()

fig, axs = plt.subplots(6, 5, figsize=(20, 12))
fig.suptitle("EEG Power Spectrum (30 ROIs)", fontsize=16)

for i in range(30):
    row, col = divmod(i, 5)
    ax = axs[row, col]
    ax.plot(np.arange(eeg_power.shape[1]), eeg_power[i].numpy())
    ax.set_ylim([eeg_ymin, eeg_ymax])
    ax.set_title(f'ROI {i}')
    ax.set_xticks([0, 25, 50])
    ax.set_xlabel("Freq bin")
    ax.set_ylabel("Power")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

### 畫 Source Power ###
src_ymin = source_power.min().item()
src_ymax = source_power.max().item()

n_sources = source_power.shape[0]
nrows = ncols = int(np.ceil(np.sqrt(n_sources)))  # 自動決定最接近平方的排列方式

fig, axs = plt.subplots(nrows, ncols, figsize=(24, 24))
fig.suptitle("Source Power Spectrum (204 Sources)", fontsize=18)

for i in range(n_sources):
    row, col = divmod(i, ncols)
    ax = axs[row, col]
    ax.plot(np.arange(source_power.shape[1]), source_power[i].numpy())
    ax.set_ylim([src_ymin, src_ymax])
    ax.set_title(f'Source {i}', fontsize=8)
    ax.set_xticks([0, 25, 50])
    ax.tick_params(labelsize=6)

# 將多餘的 subplot 清除
for j in range(n_sources, nrows * ncols):
    fig.delaxes(axs[j // ncols, j % ncols])

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


### Merge Dataset Test 

In [1]:
import sys
sys.path.insert(0, './Dataloader')
from SLT_dataloader import EEGROIDataset, SignalDataCollator, EEGROI_fft_Dataset, EEGROI_Power_Dataset, EEGROI_Merge_Dataset

data_path_1 = "G:\共用雲端硬碟\CNElab_陳昱祺\source localization\simulate_data\\dataset_seedsource_2000_1000000.0_20250404_215543\\"
data_path_2 = "G:\共用雲端硬碟\CNElab_陳昱祺\source localization\simulate_data\\dataset_seedsource_2000_10.0_20250403_161648\\"
data_path_3 = "G:\共用雲端硬碟\CNElab_陳昱祺\source localization\simulate_data\\dataset_seedsource_2000_1.0_20250403_161635\\"
test_path = "G:\\共用雲端硬碟\\CNElab_陳昱祺\\source localization\\test_data\\ROI\\Desikan_Kilianny_with_3pca\\"
group_file = "./subject_groups.json"


dataset_1 = EEGROI_Power_Dataset(data_path_1)
dataset_2 = EEGROI_Power_Dataset(data_path_2)
dataset_3 = EEGROI_Power_Dataset(data_path_3)
dataset_4 = EEGROI_fft_Dataset(test_path, test_path, group_file, "eval_dataset")

merge_dataset = EEGROI_Merge_Dataset([dataset_1, dataset_2, dataset_3, dataset_4])

<>:5: SyntaxWarning: invalid escape sequence '\C'
<>:6: SyntaxWarning: invalid escape sequence '\C'
<>:7: SyntaxWarning: invalid escape sequence '\C'
<>:5: SyntaxWarning: invalid escape sequence '\C'
<>:6: SyntaxWarning: invalid escape sequence '\C'
<>:7: SyntaxWarning: invalid escape sequence '\C'
C:\Users\user\AppData\Local\Temp\ipykernel_648\2904701036.py:5: SyntaxWarning: invalid escape sequence '\C'
  data_path_1 = "G:\共用雲端硬碟\CNElab_陳昱祺\source localization\simulate_data\\dataset_seedsource_2000_1000000.0_20250404_215543\\"
C:\Users\user\AppData\Local\Temp\ipykernel_648\2904701036.py:6: SyntaxWarning: invalid escape sequence '\C'
  data_path_2 = "G:\共用雲端硬碟\CNElab_陳昱祺\source localization\simulate_data\\dataset_seedsource_2000_10.0_20250403_161648\\"
C:\Users\user\AppData\Local\Temp\ipykernel_648\2904701036.py:7: SyntaxWarning: invalid escape sequence '\C'
  data_path_3 = "G:\共用雲端硬碟\CNElab_陳昱祺\source localization\simulate_data\\dataset_seedsource_2000_1.0_20250403_161635\\"


EEG Power shape: torch.Size([20000, 30, 100])
Source Power shape: torch.Size([20000, 68, 100])
EEG Power shape: torch.Size([20000, 30, 100])
Source Power shape: torch.Size([20000, 68, 100])
EEG Power shape: torch.Size([20000, 30, 100])
Source Power shape: torch.Size([20000, 68, 100])
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
Total NaN count for subject: 0
Total Random count for subject: 154
Total NaN count for subject: 0
Total Random count for subject: 150
Total NaN count for subject: 0
Total Random count for subject: 125
Total NaN count for subject: 0
Total Random count for subject: 152
Total NaN count for subject: 0
Total Random count for subject: 151
Total NaN count for subject: 0
Total Random count for subject: 158
Total NaN count for subject: 0
Total Random count for subject: 115
Total NaN count for subject: 0
Total Random count for subject: 117
Total NaN count for subject: 0
Total Random count for subject: 157


# Mask Data Collector

In [ ]:
from SLT_dataloader import RandonMaskDataCollator
from torch.utils.data import DataLoader, Dataset
import torch

class FakeEEGDataset(Dataset):
    def __init__(self, num_samples=3):
        self.num_samples = num_samples

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        src = torch.randn(204, 100)  # 模擬你的EEG輸入
        label = src.clone()          # tgt 是 copy（真實世界可以不一樣）
        return {"src": src, "label": label}
    
data_collator = RandonMaskDataCollator(mask_prob=0.15)

dataset = FakeEEGDataset()

loader = DataLoader(dataset, batch_size=2, collate_fn=data_collator)

for batch in loader:
    print("src shape:", batch["src"].shape)
    print("tgt shape:", batch["tgt"].shape)
    print("tgt_mask shape:", batch["tgt_token_mask"].shape)
    print("masked tokens (per sample):", batch["tgt_token_mask"].sum(dim=1))
    
    # 示範一個 mask 的效果
    i = 0  # 看第一個 sample
    mask = batch["tgt_token_mask"][i]               # shape: (204,)
    tgt = batch["tgt"][i]                     # shape: (204, 100)
    print(f"\nSample {i} -- Masked {mask.sum().item()} tokens")
    
    # 你可以選擇畫圖或手動觀察數值
    print("Masked segment [index 5]:")
    print("mask =", mask[5].item())
    print("tgt[5] =", tgt[5])
    break  # 只測一個 batch 就好


src shape: torch.Size([2, 204, 100])
tgt shape: torch.Size([2, 204, 100])
tgt_mask shape: torch.Size([2, 204])
masked tokens (per sample): tensor([31, 28])

Sample 0 -- Masked 31 tokens
Masked segment [index 5]:
mask = False
tgt[5] = tensor([-1.8127, -1.6156, -1.3425,  1.1401,  0.6811,  1.2217,  1.0362, -0.4069,
        -0.1092,  0.5771, -0.7600, -0.5149,  0.5051,  0.2441,  0.0034,  0.4732,
        -0.2585,  0.5293,  1.1682,  0.1751,  0.0125, -0.2193,  1.0801,  0.3980,
        -1.7289,  0.4181,  0.1808, -0.0203,  0.7443, -1.4836,  0.3459,  0.3904,
        -0.7059, -1.2904, -0.3906,  1.0222, -0.5746, -0.9831, -0.1352,  1.6690,
        -0.5216, -0.9523, -2.3787,  0.2937, -2.3520, -0.6608,  2.0630, -1.7844,
         0.4346,  1.2261,  0.6089, -0.0254,  0.5539, -0.7214, -1.0555,  0.2371,
        -0.3806, -0.5064, -0.8619,  1.0198,  0.8038, -0.5148,  0.0674,  0.8115,
         1.0426,  0.2588,  0.1823, -0.5281,  0.3789,  0.8928, -0.1840, -0.1180,
        -0.1063, -0.5002, -1.0513, -0.2561, -0